In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch

torch.manual_seed(1)

In [2]:
import pandas as pd

In [3]:
train = pd.read_json('/home/davankov/education/snli_1.0/snli_1.0_train.jsonl', lines=True)

In [4]:
#test = pd.read_json('/home/davankov/education/snli_1.0/snli_1.0_test.jsonl', lines=True)

In [4]:
train.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is at a diner, ordering an omelette.",( ( A person ) ( ( ( ( is ( at ( a diner ) ) )...,(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down a...,( ( ( A person ) ( on ( a horse ) ) ) ( ( jump...,(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN o...,"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ...",(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) ...
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ...,(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VB...
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at c...,(ROOT (NP (S (NP (NNP Children)) (VP (VBG smil...,There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NN...


In [5]:
new_train = train[['sentence1', 'sentence2', 'gold_label'] ]

In [6]:
new_train

,sentence1,sentence2,gold_label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,neutral
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",contradiction
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",entailment
3,Children smiling and waving at camera,They are smiling at their parents,neutral
4,Children smiling and waving at camera,There are children present,entailment
5,Children smiling and waving at camera,The kids are frowning,contradiction
6,A boy is jumping on skateboard in the middle o...,The boy skates down the sidewalk.,contradiction
7,A boy is jumping on skateboard in the middle o...,The boy does a skateboarding trick.,entailment
8,A boy is jumping on skateboard in the middle o...,The boy is wearing safety equipment.,neutral
9,An older man sits with his orange juice at a s...,An older man drinks his juice as he waits for ...,neutral


In [8]:
vocab = dict()
se = set()
i = 1
for idx, sent in new_train.iterrows():
    tokens = sent['sentence1'].lower().split(' ')
    tokens.extend(sent['sentence2'].lower().split(' '))
    for token in tokens:
        se.add(token)
        try:
            vocab[token]
        except:
            vocab[token] = i
            i+=1

In [6]:
# vocab2 = dict()
# i = 1
# for idx, sent in test.iterrows():
#     tokens = sent['sentence1'].lower().split(' ')
#     tokens.extend(sent['sentence2'].lower().split(' '))
#     for token in tokens:
#         try:
#             vocab2[token]
#         except:
#             vocab2[token] = i
#             i+=1

In [9]:
vocab['out_of_vocab'] = 0

In [10]:
len(vocab)

56257

In [11]:
vocab['sentence_padding'] = len(vocab)

In [13]:
# vocab2['out_of_vocab'] = 0
# vocab2['sentence_padding'] = len(vocab2)

In [14]:
from torch.autograd import Variable

In [64]:
# embedding = nn.Embedding(10, 10)
# # a batch of 2 samples of 4 indices each
# inputt = Variable(torch.LongTensor([[1,2,4,5, 3, 1, 1],[4,3,2,8,9, 4, 9]]))
# embedding(inputt)

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.1300 -0.9286 -0.9993 -1.4282 -0.7373 -0.0306  0.1277 -0.0047 -2.3309
 -1.2245  1.9481  1.6815  0.6686 -1.3771 -0.4192 -1.4726 -0.4775  0.5501
 -1.0577 -1.2188  0.7033  1.0838  1.4373  0.4898  1.7899  0.9227  1.4088
  0.1100 -0.2419  0.2401  1.4360 -0.0668  0.8720  0.6791 -0.4543 -0.2149
  0.0489 -1.8084  2.0058  0.9735  0.5364  0.7165  1.2615 -0.0659  2.3018
  0.1300 -0.9286 -0.9993 -1.4282 -0.7373 -0.0306  0.1277 -0.0047 -2.3309
  0.1300 -0.9286 -0.9993 -1.4282 -0.7373 -0.0306  0.1277 -0.0047 -2.3309

Columns 9 to 9 
   0.3199
 -1.1967
 -0.4567
 -1.8579
 -1.4944
  0.3199
  0.3199

(1 ,.,.) = 

Columns 0 to 8 
  -1.0577 -1.2188  0.7033  1.0838  1.4373  0.4898  1.7899  0.9227  1.4088
  0.0489 -1.8084  2.0058  0.9735  0.5364  0.7165  1.2615 -0.0659  2.3018
 -1.2245  1.9481  1.6815  0.6686 -1.3771 -0.4192 -1.4726 -0.4775  0.5501
 -1.7019 -0.1952 -1.8107 -1.9912 -0.2434 -0.5782  0.6942 -1.0025 -0.0356
 -0.3523  1.9390  0.4224  0.0572 -

In [15]:
def transformator1(row):
    s1 = row['sentence1'].lower().split(' ')
    s2 = row["sentence2"].lower().split(' ')
    l1 = []
    l2 = []
    for w in s1:
        l1.append(vocab[w])
    for w in s2:
        l2.append(vocab[w])
    ans = row['gold_label']
    
    if ans == "neutral":
        a = 0
    elif ans == "entailment":
        a = 1
    elif ans == 'contradiction':
        a = 2
    return l1

In [16]:
def transformator2(row):
    s1 = row['sentence1'].lower().split(' ')
    s2 = row["sentence2"].lower().split(' ')
    l1 = []
    l2 = []
    for w in s1:
        l1.append(vocab[w])
    for w in s2:
        l2.append(vocab[w])
    ans = row['gold_label']
    
    if ans == "neutral":
        a = 0
    elif ans == "entailment":
        a = 1
    elif ans == 'contradiction':
        a = 2
    return l2

In [17]:
def transformatorT1(row):
    s1 = row['sentence1'].lower().split(' ')
    s2 = row["sentence2"].lower().split(' ')
    l1 = []
    l2 = []
    for w in s1:
        l1.append(vocab2[w])
    for w in s2:
        l2.append(vocab2[w])
    ans = row['gold_label']
    
    if ans == "neutral":
        a = 0
    elif ans == "entailment":
        a = 1
    elif ans == 'contradiction':
        a = 2
    return l1

In [18]:
def transformatorT2(row):
    s1 = row['sentence1'].lower().split(' ')
    s2 = row["sentence2"].lower().split(' ')
    l1 = []
    l2 = []
    for w in s1:
        l1.append(vocab2[w])
    for w in s2:
        l2.append(vocab2[w])
    ans = row['gold_label']
    
    if ans == "neutral":
        a = 0
    elif ans == "entailment":
        a = 1
    elif ans == 'contradiction':
        a = 2
    return l2

In [19]:
df = pd.DataFrame()

In [20]:
df["S2"] = new_train.apply(transformator2, axis=1)

In [21]:
df["S1"] = new_train.apply(transformator1, axis=1)

In [22]:
def func(x):
    if x == 'contradiction':
        return 2
    elif x == 'neutral':
        return 0
    elif x == "entailment":
        return 1

In [23]:
df['ans'] = train['gold_label'].apply(func)

In [24]:
df.head()

,S2,S1,ans
0,"[1, 2, 10, 11, 12, 4, 13, 1, 14]","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]",0.0
1,"[1, 2, 10, 15, 1, 16, 17, 18, 19]","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]",2.0
2,"[1, 2, 10, 20, 3, 1, 21]","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]",1.0
3,"[27, 28, 23, 15, 29, 30]","[22, 23, 24, 25, 15, 26]",0.0
4,"[31, 28, 22, 32]","[22, 23, 24, 25, 15, 26]",1.0


In [11]:
# new_test = test[['sentence1', 'sentence2', 'gold_label'] ]

In [12]:
# df_test = pd.DataFrame()

In [13]:
# df_test["S2"] = new_test.apply(transformatorT2, axis=1)
# df_test["S1"] = new_test.apply(transformatorT1, axis=1)

In [14]:
# df_test['ans'] = new_test['gold_label'].apply(func)

In [25]:
# def new_f(x):
#     needed = 50 - len(x)
#     x.extend([vocab["sentence_padding"] for i in range(needed)])
#     return x

In [16]:
# df_test.S1.apply(new_fT)
# df_test.S2.apply(new_fT)

0       [6, 2, 18, 19, 20, 6, 21, 8909, 8909, 8909, 89...
1       [6, 2, 22, 23, 24, 25, 8909, 8909, 8909, 8909,...
2       [16, 3, 26, 15, 16, 27, 28, 8909, 8909, 8909, ...
3       [6, 29, 22, 38, 8909, 8909, 8909, 8909, 8909, ...
4       [6, 29, 22, 35, 39, 8909, 8909, 8909, 8909, 89...
5       [6, 29, 18, 40, 41, 8909, 8909, 8909, 8909, 89...
6       [16, 44, 46, 20, 47, 48, 42, 50, 8909, 8909, 8...
7       [16, 44, 46, 20, 47, 48, 42, 51, 52, 53, 8909,...
8       [16, 44, 54, 55, 42, 50, 8909, 8909, 8909, 890...
9       [6, 56, 22, 64, 34, 65, 66, 67, 58, 68, 22, 69...
10      [71, 22, 16, 56, 58, 72, 73, 65, 74, 5, 61, 75...
11      [77, 48, 65, 66, 78, 79, 6, 80, 8909, 8909, 89...
12      [16, 81, 82, 22, 87, 88, 8, 68, 89, 16, 86, 89...
13      [16, 90, 22, 91, 16, 86, 8909, 8909, 8909, 890...
14      [16, 92, 22, 93, 20, 6, 94, 48, 16, 86, 8909, ...
15      [16, 44, 95, 99, 69, 6, 100, 8909, 8909, 8909,...
16      [16, 44, 95, 97, 69, 98, 8909, 8909, 8909, 890...
17      [16, 4

In [17]:
# df_test.head()

,S2,S1,ans
0,"[6, 2, 18, 19, 20, 6, 21, 8909, 8909, 8909, 89...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 6,...",0.0
1,"[6, 2, 22, 23, 24, 25, 8909, 8909, 8909, 8909,...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 6,...",1.0
2,"[16, 3, 26, 15, 16, 27, 28, 8909, 8909, 8909, ...","[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 6,...",2.0
3,"[6, 29, 22, 38, 8909, 8909, 8909, 8909, 8909, ...","[16, 29, 24, 16, 30, 31, 32, 33, 34, 16, 35, 3...",0.0
4,"[6, 29, 22, 35, 39, 8909, 8909, 8909, 8909, 89...","[16, 29, 24, 16, 30, 31, 32, 33, 34, 16, 35, 3...",1.0


In [30]:
# df.to_csv('/home/davankov/train_di.csv')

In [4]:
# df = pd.read_csv('/home/davankov/train_di.csv')

In [26]:
def new_f(x):
    needed = 50 - len(x)
    x.extend([vocab["sentence_padding"] for i in range(needed)])
    return x

In [27]:
df.S1.apply(new_f)
df.S2.apply(new_f)

0         [1, 2, 10, 11, 12, 4, 13, 1, 14, 56257, 56257,...
1         [1, 2, 10, 15, 1, 16, 17, 18, 19, 56257, 56257...
2         [1, 2, 10, 20, 3, 1, 21, 56257, 56257, 56257, ...
3         [27, 28, 23, 15, 29, 30, 56257, 56257, 56257, ...
4         [31, 28, 22, 32, 56257, 56257, 56257, 56257, 5...
5         [33, 34, 28, 35, 56257, 56257, 56257, 56257, 5...
6         [33, 36, 44, 8, 33, 45, 56257, 56257, 56257, 5...
7         [33, 36, 46, 1, 47, 48, 56257, 56257, 56257, 5...
8         [33, 36, 10, 49, 50, 51, 56257, 56257, 56257, ...
9         [18, 52, 53, 69, 12, 57, 70, 71, 72, 13, 12, 7...
10        [1, 36, 78, 1, 79, 56257, 56257, 56257, 56257,...
11        [18, 80, 53, 54, 39, 1, 58, 81, 56257, 56257, ...
12        [88, 84, 28, 85, 3, 89, 56257, 56257, 56257, 5...
13        [33, 84, 28, 90, 56257, 56257, 56257, 56257, 5...
14        [31, 28, 84, 91, 92, 56257, 56257, 56257, 5625...
15        [33, 94, 28, 100, 101, 56257, 56257, 56257, 56...
16        [33, 94, 28, 102, 15, 103, 39,

In [5]:
# train_df  = df[df.ans.apply(lambda x: True if x == 1 or x == 0 or x == 2 else False)]

In [45]:
# train_df.shape

(549367, 3)

In [14]:
# train_df.ans = train_df.S1.apply(lambda x: list(x))

/home/davankov/.local/lib/python3.5/site-packages/pandas/core/generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [16]:
# train_df.head()

,Unnamed: 0,S1,S2,ans
0,0,"[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]","[[, 1, ,, , 2, ,, , 3, ,, , 1, ,, , 4, ,, ..."
1,1,"[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]","[[, 1, ,, , 2, ,, , 3, ,, , 1, ,, , 4, ,, ..."
2,2,"[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9]","[[, 1, ,, , 2, ,, , 3, ,, , 1, ,, , 4, ,, ..."
3,3,"[22, 23, 24, 25, 15, 26]","[22, 23, 24, 25, 15, 26]","[[, 2, 2, ,, , 2, 3, ,, , 2, 4, ,, , 2, 5, ..."
4,4,"[22, 23, 24, 25, 15, 26]","[22, 23, 24, 25, 15, 26]","[[, 2, 2, ,, , 2, 3, ,, , 2, 4, ,, , 2, 5, ..."


In [28]:
df

,S2,S1,ans
0,"[1, 2, 10, 11, 12, 4, 13, 1, 14, 56257, 56257,...","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9, 56257, 56257...",0.0
1,"[1, 2, 10, 15, 1, 16, 17, 18, 19, 56257, 56257...","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9, 56257, 56257...",2.0
2,"[1, 2, 10, 20, 3, 1, 21, 56257, 56257, 56257, ...","[1, 2, 3, 1, 4, 5, 6, 1, 7, 8, 9, 56257, 56257...",1.0
3,"[27, 28, 23, 15, 29, 30, 56257, 56257, 56257, ...","[22, 23, 24, 25, 15, 26, 56257, 56257, 56257, ...",0.0
4,"[31, 28, 22, 32, 56257, 56257, 56257, 56257, 5...","[22, 23, 24, 25, 15, 26, 56257, 56257, 56257, ...",1.0
5,"[33, 34, 28, 35, 56257, 56257, 56257, 56257, 5...","[22, 23, 24, 25, 15, 26, 56257, 56257, 56257, ...",2.0
6,"[33, 36, 44, 8, 33, 45, 56257, 56257, 56257, 5...","[1, 36, 10, 37, 3, 38, 39, 33, 40, 41, 1, 42, ...",2.0
7,"[33, 36, 46, 1, 47, 48, 56257, 56257, 56257, 5...","[1, 36, 10, 37, 3, 38, 39, 33, 40, 41, 1, 42, ...",1.0
8,"[33, 36, 10, 49, 50, 51, 56257, 56257, 56257, ...","[1, 36, 10, 37, 3, 38, 39, 33, 40, 41, 1, 42, ...",0.0
9,"[18, 52, 53, 69, 12, 57, 70, 71, 72, 13, 12, 7...","[18, 52, 53, 54, 55, 12, 56, 57, 15, 1, 58, 59...",0.0


In [29]:
from torch.utils import data
import torch.nn.functional as F

In [30]:
batch_size = 49
hidden_size = 70

In [32]:
x = Variable(torch.LongTensor(df.S1.loc[:48]))
y = Variable(torch.LongTensor(df.S2.loc[:48]))

In [35]:
class Model(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(len(vocab) + 20, hidden_size)
        self.convs = nn.Sequential(
            nn.Conv1d(hidden_size, hidden_size, 3, padding=1),
            nn.Tanh(),
        )
        self.lin = nn.Linear(2*hidden_size, 3)
    def forward(self, x, y):
        res_x = self.convs(self.embedding(x).transpose(1, 2))
        res_y = self.convs(self.embedding(y).transpose(1, 2))
        vec_x = torch.squeeze(F.max_pool1d(res_x, res_x.size(1)))
        vec_y = torch.squeeze(F.max_pool1d(res_y, res_y.size(1)))
        result = torch.cat([vec_x, vec_y], dim=1)
        return self.lin(result)

In [36]:
cnn =  Model(hidden_size)

In [37]:
x.size(), y.size()

(torch.Size([49, 50]), torch.Size([49, 50]))

In [ ]:
cnn(x, y)